In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinchilla/sardana24a.pdf


In [43]:
## Reference notbook - https://www.kaggle.com/code/iamrahulthorat/claude-demo/notebook

In [8]:
!pip install anthropic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 3.5 MB/s eta 0:00:0000:0100:01


In [ ]:
ANTHROPIC_API_KEY='key'

In [12]:
import anthropic

# Setting up the api key
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("claude-api-key")

client = anthropic.Anthropic(api_key=secret_value_0)

In [24]:
opus = 'claude-3-opus-20240229'     # Most powerful model, most expensive
sonnet = 'claude-3-sonnet-20240229' # Most balanced model between intelligence and speed
haiku = 'claude-3-haiku-20240307'   # Fastest and most compact model
opus4= "claude-opus-4-20250514"

In [26]:
def model_request(model1="claude-opus-4-20250514", system1="You are a world-class poet. Respond only with short poems.", prompt1="Why is the ocean salty?"):
    message = client.messages.create(
        model= model1, #"claude-opus-4-20250514",
        max_tokens=1000,
        temperature=1,
        system= system1, #"You are a world-class poet. Respond only with short poems.",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":  prompt1 #"Why is the ocean salty?"
                    }
                ]
            }
        ])
    
    return message.content[0].text

res = model_request()
print (res)

Ancient rivers weep and flow,
Carrying earth's minerals below.
Into vast basins, waters creep,
Depositing salt they cannot keep.

Sun lifts water to the sky,
Pure vapor rises, salt stays dry.
Eons pass, the cycle spins—
Salt remains while water thins.


## text extraction

In [30]:
prompt = '''Precisely extract any email addresses from the following text and then write them, one per line.
            Only write an email address if it's precisely spelled out in the input text.
            If there are no email addresses in the text, write "N/A". Do not say anything else.

            Phone Directory:
            Thomas Shelby, 555-123-4567, [thomas.shelby@peakyblinders.com]
            Arthur Shelby, 555-987-6543, [arthur.shelby@peakyblinders.com]
            Polly Gray, 555-345-6789, [polly.gray@peakyblinders.com]

            Phone directory will be kept up to date by the HR manager.'''

res = model_request(system1="", prompt1=prompt)
print (res)

thomas.shelby@peakyblinders.com
arthur.shelby@peakyblinders.com
polly.gray@peakyblinders.com


In [31]:
res = model_request(model1 = sonnet , system1="" ,prompt1=prompt)
print (res)

/tmp/ipykernel_35/3019808832.py:2: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = client.messages.create(


thomas.shelby@peakyblinders.com
arthur.shelby@peakyblinders.com
polly.gray@peakyblinders.com


## PDF extraction

In [35]:
!pip install PyPDF2 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00a 0:00:01


In [37]:
from PyPDF2 import PdfReader

In [38]:
with open("/kaggle/input/chinchilla/sardana24a.pdf", "rb") as file:
    reader = PdfReader(file)
    num_of_pages = len(reader.pages)
    case_of_identity = "".join(page.extract_text() for page in reader.pages)
    print (case_of_identity[:1000])

Beyond Chinchilla-Optimal:
Accounting for Inference in Language Model Scaling Laws
Nikhil Sardana1Jacob Portes1Sasha Doubov1Jonathan Frankle1
Abstract
Large language model (LLM) scaling laws are em-
pirical formulas that estimate changes in model
quality as a result of increasing parameter count
and training data. However, these formulas, in-
cluding the popular Deepmind Chinchilla scaling
laws, neglect to include the cost of inference. We
modify the Chinchilla scaling laws to calculate the
optimal LLM parameter count and pre-training
data size to train and deploy a model of a given
quality and inference demand. We conduct our
analysis both in terms of a compute budget and
real-world costs and find that LLM researchers ex-
pecting reasonably large inference demand ( ⁄tildelow1B
requests) should train models smaller and longer
than Chinchilla-optimal. Furthermore, we train
47 models of varying sizes and parameter counts
to validate our formula and find that model quality
continues to im

In [40]:
prompt = 'Summarize the following case of identity story in 2 sentences: ' + case_of_identity

# Call the function with the inputs
res = model_request(model1=sonnet, system1= "",  prompt1=prompt)
print (res)

/tmp/ipykernel_35/3019808832.py:2: DeprecationWarning: The model 'claude-3-sonnet-20240229' is deprecated and will reach end-of-life on July 21st, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  message = client.messages.create(


Here are two sentences summarizing the key points about accounting for inference in language model scaling laws:

1) The paper modifies the Chinchilla scaling laws to account for the computational costs of inference, showing that for high inference demand, it is optimal to train smaller models on more data than prescribed by the original Chinchilla scaling laws. 

2) Experiments training models at extreme token/parameter ratios up to 10,000 show that model quality as measured by loss and downstream metrics continues improving with more data, contradicting the hypothesis that there is a "critical size" below which models cannot match larger Chinchilla-style models.


### PRompt Caching

In [44]:
# import anthropic

# client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    system=[
      {
        "type": "text",
        "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n",
      },
      {
        "type": "text",
        "text": "<the entire contents of 'Pride and Prejudice'>",
        "cache_control": {"type": "ephemeral"}
      }
    ],
    messages=[{"role": "user", "content": "Analyze the major themes in 'Pride and Prejudice'."}],
)
print(response.usage.model_dump_json())

# Call the model again with the same inputs up to the cache checkpoint
response = client.messages.create(
     model="claude-opus-4-20250514",
    max_tokens=1024,
    system=[
      {
        "type": "text",
        "text": "You are an AI assistant tasked with analyzing literary works. Your goal is to provide insightful commentary on themes, characters, and writing style.\n",
      },
     
    ],
    messages=[{"role": "user", "content": "Analyze the major themes in 'Pride and Prejudice'."}],

)
print(response.usage.model_dump_json())

{"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"input_tokens":65,"output_tokens":531,"server_tool_use":null,"service_tier":"standard"}
{"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"input_tokens":51,"output_tokens":463,"server_tool_use":null,"service_tier":"standard"}


In [45]:
# import anthropic

# client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    messages=[{
        "role": "user",
        "content": "Are there an infinite number of prime numbers such that n mod 4 == 3?"
    }]
)

# The response will contain summarized thinking blocks and text blocks
for block in response.content:
    if block.type == "thinking":
        print(f"\nThinking summary: {block.thinking}")
    elif block.type == "text":
        print(f"\nResponse: {block.text}")


Thinking summary: This is asking about primes that are congruent to 3 modulo 4, i.e., primes of the form 4k + 3.

Let me think about this systematically. I need to prove that there are infinitely many primes p such that p ≡ 3 (mod 4).

This is a classic result in number theory. Let me try to prove it using a proof by contradiction, similar to Euclid's proof for the infinitude of primes, but with a twist.

Assume for contradiction that there are only finitely many primes congruent to 3 modulo 4. Let's call them p₁, p₂, ..., pₖ.

Now I need to construct a number that must have a prime factor congruent to 3 modulo 4, but which is not divisible by any of p₁, p₂, ..., pₖ.

Consider the number N = 4(p₁p₂...pₖ) - 1.

Note that N ≡ -1 ≡ 3 (mod 4).

Now, N is not divisible by any of p₁, p₂, ..., pₖ because:
- If pᵢ divides N, then pᵢ divides 4(p₁p₂...pₖ) - 1
- This means 4(p₁p₂...pₖ) ≡ 1 (mod pᵢ)
- Since pᵢ divides p₁p₂...pₖ, we have p₁p₂...pₖ ≡ 0 (mod pᵢ)
- So 4 · 0 ≡ 1 (mod pᵢ), which means 

In [46]:
response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "The grass is green. The sky is blue."
                    },
                    "title": "My Document",
                    "context": "This is a trustworthy document.",
                    "citations": {"enabled": True}
                },
                {
                    "type": "text",
                    "text": "What color is the grass and sky?"
                }
            ]
        }
    ]
)
print(response)

Message(id='msg_01M19euXE7ukGQ5uCn5ASief', content=[TextBlock(citations=None, text='Based on the document provided:\n\n', type='text'), TextBlock(citations=[CitationCharLocation(cited_text='The grass is green. ', document_index=0, document_title='My Document', end_char_index=20, start_char_index=0, type='char_location')], text='The grass is green.', type='text'), TextBlock(citations=None, text=' ', type='text'), TextBlock(citations=[CitationCharLocation(cited_text='The sky is blue.', document_index=0, document_title='My Document', end_char_index=36, start_char_index=20, type='char_location')], text='The sky is blue.', type='text')], model='claude-opus-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=610, output_tokens=52, server_tool_use=None, service_tier='standard'))


In [47]:
response.content

[TextBlock(citations=None, text='Based on the document provided:\n\n', type='text'),
 TextBlock(citations=[CitationCharLocation(cited_text='The grass is green. ', document_index=0, document_title='My Document', end_char_index=20, start_char_index=0, type='char_location')], text='The grass is green.', type='text'),
 TextBlock(citations=None, text=' ', type='text'),
 TextBlock(citations=[CitationCharLocation(cited_text='The sky is blue.', document_index=0, document_title='My Document', end_char_index=36, start_char_index=20, type='char_location')], text='The sky is blue.', type='text')]

In [48]:
# Long document content (e.g., technical documentation)
long_document = "This is a very long document with thousands of words..." + " ... " * 1000  # Minimum cacheable length

response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": long_document
                    },
                    "citations": {"enabled": True},
                    "cache_control": {"type": "ephemeral"}  # Cache the document content
                },
                {
                    "type": "text",
                    "text": "What does this document say about API features?"
                }
            ]
        }
    ]
)
print (response)

Message(id='msg_019tRK49v4QyRJbTTADdswG6', content=[TextBlock(citations=None, text='I\'ve reviewed the document you provided, but I cannot find any information about API features in it. The document appears to contain only placeholder text ("This is a very long document with thousands of words..." followed by many ellipses), rather than actual content about APIs or their features.\n\nIf you have a different document that contains information about API features, please share that instead, and I\'ll be happy to help you analyze it.', type='text')], model='claude-opus-4-20250514', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=2548, cache_read_input_tokens=0, input_tokens=13, output_tokens=90, server_tool_use=None, service_tier='standard'))


In [ ]:
# Initial request with web search
response = client.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Search for comprehensive information about quantum computing breakthroughs in 2025"
        }
    ],
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 10
    }]
)

# Check if the response has pause_turn stop reason
if response.stop_reason == "pause_turn":
    # Continue the conversation with the paused content
    messages = [
        {"role": "user", "content": "Search for comprehensive information about quantum computing breakthroughs in 2025"},
        {"role": "assistant", "content": response.content}
    ]
    
    # Send the continuation request
    continuation = client.messages.create(
        model="claude-3-7-sonnet-latest",
        max_tokens=1024,
        messages=messages,
        tools=[{
            "type": "web_search_20250305",
            "name": "web_search",
            "max_uses": 10
        }]
    )
    
    print(continuation)
else:
    print(response)
